In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the custom model architecture
class Block(nn.Module):
    def __init__(self, in_channels, k):
        super(Block, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(in_channels, k)
        self.relu = nn.ReLU()
        self.convs = nn.ModuleList([nn.Conv2d(in_channels, in_channels, 3, padding=1) for _ in range(k)])

    def forward(self, x):
        a = self.pool(x)
        a = a.view(a.size(0), -1)
        a = self.relu(self.linear(a))
        out = sum(a[:, i].view(-1, 1, 1, 1) * self.convs[i](x) for i in range(len(self.convs)))
        return out


class CIFAR10Classifier(nn.Module):
    def __init__(self, num_blocks, in_channels, k):
        super(CIFAR10Classifier, self).__init__()
        self.backbone = nn.Sequential(*[Block(in_channels, k) for _ in range(num_blocks)])
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(in_channels, 10)

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4),
                                transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Initialize and train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CIFAR10Classifier(num_blocks=3, in_channels=3, k=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

for epoch in range(5): #200
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

# Test the model and calculate accur


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 2.3059030604362487
Epoch 2, Loss: 2.3047655358314514
Epoch 3, Loss: 2.3050659856796263
Epoch 4, Loss: 2.3048075804710386
Epoch 5, Loss: 2.3050419726371767


In [8]:
# Function to compute the accuracy
def accuracy(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Test the model and calculate accuracies
model.eval()
train_accuracy = accuracy(model, trainloader)
test_accuracy = accuracy(model, testloader)
print(f"Train accuracy: {train_accuracy}%")
print(f"Test accuracy: {test_accuracy}%")


Train accuracy: 10.0%
Test accuracy: 10.0%


In [13]:
print(accuracy(model, trainloader))

10.0


In [9]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.9 MB/s eta 0:00:00


In [ ]:
"""
# Get the best hyperparameters
best_params = study.best_params
lr = best_params["lr"]
momentum = best_params["momentum"]
weight_decay = best_params["weight_decay"]
num_blocks = best_params["num_blocks"]
k = best_params["k"]

# Train the model with the best hyperparameters on the entire training set
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
model = CIFAR10Classifier(num_blocks=num_blocks, in_channels=3, k=k).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

for epoch in range(10):  # You can adjust the number of epochs
    train_loss = train(model, trainloader, optimizer, device)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

# Calculate train and test accuracies
model.eval()
train_accuracy = accuracy(model, trainloader)
test_accuracy = accuracy(model, testloader)
print(f"Train accuracy: {train_accuracy}%")
print(f"Test accuracy: {test_accuracy}%")
"""


In [11]:
import optuna

# Create a validation set
val_size = int(0.1 * len(trainset))
train_size = len(trainset) - val_size
train_subset, val_subset = torch.utils.data.random_split(trainset, [train_size, val_size])
trainloader = torch.utils.data.DataLoader(train_subset, batch_size=100, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_subset, batch_size=100, shuffle=False, num_workers=2)

# Define the training function
def train(model, trainloader, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(trainloader)

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    momentum = trial.suggest_uniform("momentum", 0.5, 0.9)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)
    num_blocks = trial.suggest_int("num_blocks", 2, 6)
    k = trial.suggest_int("k", 2, 10)

    # Initialize and train the model
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = CIFAR10Classifier(num_blocks=num_blocks, in_channels=3, k=k).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    
    for epoch in range(10):
        train_loss = train(model, trainloader, optimizer, device)
        val_accuracy = accuracy(model, valloader)
        trial.report(val_accuracy, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.TrialPruned()

    return val_accuracy

# Optimize the objective function
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=2) #50

# Print the best hyperparameters
print("Best hyperparameters:", study.best_params)


[I 2023-04-08 23:53:41,284] A new study created in memory with name: no-name-3c26ba1e-4489-4e2d-ac81-56fe984aa067
<ipython-input-11-51bcd98187ad>:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
<ipython-input-11-51bcd98187ad>:28: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  momentum = trial.suggest_uniform("momentum", 0.5, 0.9)
<ipython-input-11-51bcd98187ad>:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  weight_decay = tri

Best hyperparameters: {'lr': 0.00010536716571160901, 'momentum': 0.8681030412473103, 'weight_decay': 0.0008984423699455039, 'num_blocks': 4, 'k': 7}


In [12]:
# Get the best hyperparameters
best_params = study.best_params
lr = best_params["lr"]
momentum = best_params["momentum"]
weight_decay = best_params["weight_decay"]
num_blocks = best_params["num_blocks"]
k = best_params["k"]

# Train the model with the best hyperparameters on the entire training set
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
model = CIFAR10Classifier(num_blocks=num_blocks, in_channels=3, k=k).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

for epoch in range(10):  # You can adjust the number of epochs
    train_loss = train(model, trainloader, optimizer, device)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

# Calculate train and test accuracies
model.eval()
train_accuracy = accuracy(model, trainloader)
test_accuracy = accuracy(model, testloader)
print(f"Train accuracy: {train_accuracy}%")
print(f"Test accuracy: {test_accuracy}%")

Epoch 1, Loss: 2.3656892743110656
Epoch 2, Loss: 2.360637814998627
Epoch 3, Loss: 2.356008689880371
Epoch 4, Loss: 2.3517649097442628
Epoch 5, Loss: 2.347868968963623
Epoch 6, Loss: 2.3442946271896363
Epoch 7, Loss: 2.3410070796012876
Epoch 8, Loss: 2.3379897079467775
Epoch 9, Loss: 2.3352128677368165
Epoch 10, Loss: 2.332660102367401
Train accuracy: 10.0%
Test accuracy: 10.0%


In [14]:
# Get the best hyperparameters
best_params = study.best_params
lr = 0.010536716571160901
momentum = 0.5
weight_decay = 3
num_blocks = 3
k = 1

# Train the model with the best hyperparameters on the entire training set
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
model = CIFAR10Classifier(num_blocks=num_blocks, in_channels=3, k=k).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

for epoch in range(10):  # You can adjust the number of epochs
    train_loss = train(model, trainloader, optimizer, device)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

# Calculate train and test accuracies
model.eval()
train_accuracy = accuracy(model, trainloader)
test_accuracy = accuracy(model, testloader)
print(f"Train accuracy: {train_accuracy}%")
print(f"Test accuracy: {test_accuracy}%")

Epoch 1, Loss: 2.303606457233429
Epoch 2, Loss: 2.3025970883369444
Epoch 3, Loss: 2.302597803115845
Epoch 4, Loss: 2.302601987838745
Epoch 5, Loss: 2.302602737903595
Epoch 6, Loss: 2.302606696128845
Epoch 7, Loss: 2.302599596500397
Epoch 8, Loss: 2.302606887340546
Epoch 9, Loss: 2.3025857496261595
Epoch 10, Loss: 2.302589078426361
Train accuracy: 10.0%
Test accuracy: 10.0%
